# 🚨 BVMT Anomaly Detection System
## Market Surveillance & Alert Generation

**Objective:** Detect suspicious market activity in BVMT stocks

**Detection Types:**
- Volume spikes (>3 standard deviations)
- Abnormal price movements (>5% without news)
- Unusual trading patterns

**Model:** Isolation Forest + Rule-Based Thresholds (Hybrid Approach)

## 1. Setup & Data Loading

In [1]:
!pip install -q scikit-learn pandas numpy matplotlib seaborn joblib

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = (14, 6)

print("✅ Libraries imported")

✅ Libraries imported


In [ ]:
# Upload data files
from google.colab import files
uploaded = files.upload()

In [9]:
# Load data (reuse from price prediction notebook)
def load_csv_file(filepath):
    df = pd.read_csv(filepath, sep=';', encoding='utf-8')
    df.columns = df.columns.str.strip()
    return df

def load_all_recent_data():
    """Load 2022-2025 data for anomaly detection (more recent = more relevant)"""
    all_dfs = []
    for year in range(2022, 2026):
        try:
            df = load_csv_file(f'/content/histo_cotation_{year}.csv')
            all_dfs.append(df)
            print(f"✅ Loaded {year}: {len(df)} rows")
        except Exception as e:
            print(f"⚠️ Could not load {year}: {e}")

    return pd.concat(all_dfs, ignore_index=True)

raw_data = load_all_recent_data()

✅ Loaded 2022: 83939 rows
✅ Loaded 2023: 147209 rows
✅ Loaded 2024: 145381 rows
✅ Loaded 2025: 142853 rows


In [10]:
def clean_data(df):
    df = df.copy()

    # Debug: Check SEANCE format first
    print(f"🔍 SEANCE sample: {df['SEANCE'].head(3).tolist()}")
    print(f"   SEANCE dtype: {df['SEANCE'].dtype}")

    # Parse date - let pandas infer the format automatically
    df['date'] = pd.to_datetime(df['SEANCE'], errors='coerce')

    # If auto-detection fails, try common formats
    if df['date'].isna().all():
        for fmt in ['%Y-%m-%d', '%d-%m-%Y', '%m/%d/%Y', '%d/%m/%Y', '%Y%m%d']:
            df['date'] = pd.to_datetime(df['SEANCE'], format=fmt, errors='coerce')
            if df['date'].notna().any():
                print(f"   ✅ Date format detected: {fmt}")
                break

    print(f"📅 Valid dates: {df['date'].notna().sum()}")

    # Filter to main market
    df['GROUPE'] = pd.to_numeric(df['GROUPE'], errors='coerce')
    df = df[df['GROUPE'] == 11].copy()

    # Rename columns
    df = df.rename(columns={
        'VALEUR': 'stock_name',
        'CODE': 'stock_code',
        'OUVERTURE': 'open',
        'CLOTURE': 'close',
        'PLUS_BAS': 'low',
        'PLUS_HAUT': 'high',
        'QUANTITE_NEGOCIEE': 'volume',
        'NB_TRANSACTION': 'transactions',
        'CAPITAUX': 'capital'
    })

    # Convert numeric
    for col in ['open', 'close', 'low', 'high', 'volume', 'transactions', 'capital']:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Filter valid data
    df = df.dropna(subset=['date', 'close'])
    df = df[(df['close'] > 0) & (df['volume'] > 0)]

    return df.sort_values(['stock_name', 'date']).reset_index(drop=True)

data = clean_data(raw_data)
print(f"✅ Cleaned data: {len(data)} rows")

🔍 SEANCE sample: ['03/01/2022   ', '03/01/2022   ', '03/01/2022   ']
   SEANCE dtype: object
📅 Valid dates: 204601
✅ Cleaned data: 16053 rows


## 2. Anomaly Feature Engineering

In [11]:
def create_anomaly_features(df):
    """
    Create features specifically for anomaly detection.
    These features capture unusual market behavior.
    """
    df = df.copy().sort_values('date').reset_index(drop=True)

    # Volume-based anomaly features
    df['volume_ma_20'] = df['volume'].rolling(20).mean()
    df['volume_std_20'] = df['volume'].rolling(20).std()
    df['volume_zscore'] = (df['volume'] - df['volume_ma_20']) / df['volume_std_20']
    df['volume_ratio'] = df['volume'] / df['volume_ma_20']

    # Price-based anomaly features
    df['price_change'] = df['close'].pct_change()
    df['price_change_abs'] = df['price_change'].abs()
    df['price_ma_20'] = df['close'].rolling(20).mean()
    df['price_std_20'] = df['close'].rolling(20).std()
    df['price_zscore'] = (df['close'] - df['price_ma_20']) / df['price_std_20']

    # Intraday range (high volatility indicator)
    df['intraday_range'] = (df['high'] - df['low']) / df['close']
    df['range_ma_10'] = df['intraday_range'].rolling(10).mean()
    df['range_ratio'] = df['intraday_range'] / df['range_ma_10']

    # Gap analysis (opening gap from previous close)
    df['gap_open'] = (df['open'] - df['close'].shift(1)) / df['close'].shift(1)
    df['gap_open_abs'] = df['gap_open'].abs()

    # Transaction anomaly
    df['tx_ma_20'] = df['transactions'].rolling(20).mean()
    df['tx_ratio'] = df['transactions'] / df['tx_ma_20']

    # Volume-Price divergence (high volume but small price change = suspicious)
    df['vol_price_ratio'] = df['volume_ratio'] / (df['price_change_abs'] + 0.001)

    return df

# Apply to each stock
featured_data = []
for stock in data['stock_name'].unique():
    stock_df = data[data['stock_name'] == stock].copy()
    if len(stock_df) >= 30:  # Need minimum history
        stock_df = create_anomaly_features(stock_df)
        featured_data.append(stock_df)

data_with_features = pd.concat(featured_data, ignore_index=True)
data_with_features = data_with_features.dropna()
print(f"✅ Feature-enriched data: {len(data_with_features)} rows")

✅ Feature-enriched data: 14869 rows


## 3. Rule-Based Detection (Fast & Interpretable)

In [12]:
def detect_volume_spike(row, threshold=3.0):
    """Detect if volume is abnormally high (> threshold standard deviations)"""
    return row['volume_zscore'] > threshold

def detect_price_anomaly(row, threshold=0.05):
    """Detect if price change is abnormally high (> threshold %)"""
    return row['price_change_abs'] > threshold

def detect_gap_anomaly(row, threshold=0.03):
    """Detect if opening gap is abnormal (> threshold %)"""
    return row['gap_open_abs'] > threshold

def detect_range_anomaly(row, threshold=2.0):
    """Detect if intraday range is abnormal"""
    return row['range_ratio'] > threshold

# Apply rule-based detection
df = data_with_features.copy()
df['is_volume_spike'] = df.apply(detect_volume_spike, axis=1)
df['is_price_anomaly'] = df.apply(detect_price_anomaly, axis=1)
df['is_gap_anomaly'] = df.apply(detect_gap_anomaly, axis=1)
df['is_range_anomaly'] = df.apply(detect_range_anomaly, axis=1)

# Count anomalies per type
print("📊 Rule-Based Anomaly Detection Results:")
print(f"  Volume Spikes: {df['is_volume_spike'].sum()} ({df['is_volume_spike'].mean()*100:.2f}%)")
print(f"  Price Anomalies: {df['is_price_anomaly'].sum()} ({df['is_price_anomaly'].mean()*100:.2f}%)")
print(f"  Gap Anomalies: {df['is_gap_anomaly'].sum()} ({df['is_gap_anomaly'].mean()*100:.2f}%)")
print(f"  Range Anomalies: {df['is_range_anomaly'].sum()} ({df['is_range_anomaly'].mean()*100:.2f}%)")

📊 Rule-Based Anomaly Detection Results:
  Volume Spikes: 440 (2.96%)
  Price Anomalies: 6941 (46.68%)
  Gap Anomalies: 9249 (62.20%)
  Range Anomalies: 1498 (10.07%)


## 4. Isolation Forest (ML-Based Detection)

In [13]:
# Features for Isolation Forest
ANOMALY_FEATURES = [
    'volume_zscore',
    'volume_ratio',
    'price_change_abs',
    'price_zscore',
    'intraday_range',
    'range_ratio',
    'gap_open_abs',
    'tx_ratio',
    'vol_price_ratio'
]

# Prepare data
X = df[ANOMALY_FEATURES].values
X = np.nan_to_num(X, nan=0, posinf=0, neginf=0)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"✅ Prepared {len(X)} samples with {len(ANOMALY_FEATURES)} features")

✅ Prepared 14869 samples with 9 features


In [14]:
# Train Isolation Forest
# contamination = expected proportion of anomalies (1-2% is typical for market data)
iso_forest = IsolationForest(
    n_estimators=200,
    max_samples='auto',
    contamination=0.02,  # Expect 2% anomalies
    random_state=42,
    n_jobs=-1
)

# Fit and predict
df['iso_forest_label'] = iso_forest.fit_predict(X_scaled)
df['iso_forest_score'] = iso_forest.decision_function(X_scaled)

# -1 = anomaly, 1 = normal
df['is_ml_anomaly'] = df['iso_forest_label'] == -1

print(f"\n📊 Isolation Forest Results:")
print(f"  ML Anomalies: {df['is_ml_anomaly'].sum()} ({df['is_ml_anomaly'].mean()*100:.2f}%)")


📊 Isolation Forest Results:
  ML Anomalies: 298 (2.00%)


In [15]:
# Combine rule-based and ML detection (hybrid approach)
df['anomaly_score'] = (
    df['is_volume_spike'].astype(int) * 0.3 +
    df['is_price_anomaly'].astype(int) * 0.3 +
    df['is_gap_anomaly'].astype(int) * 0.1 +
    df['is_range_anomaly'].astype(int) * 0.1 +
    df['is_ml_anomaly'].astype(int) * 0.2
)

# Classify severity
def classify_severity(score):
    if score >= 0.6:
        return 'HIGH'
    elif score >= 0.3:
        return 'MEDIUM'
    elif score > 0:
        return 'LOW'
    return 'NONE'

df['severity'] = df['anomaly_score'].apply(classify_severity)

# Summary
print("\n📊 Anomaly Severity Distribution:")
print(df['severity'].value_counts())


📊 Anomaly Severity Distribution:
severity
MEDIUM    6769
NONE      4974
LOW       2757
HIGH       369
Name: count, dtype: int64


## 5. Visualize Anomalies

In [19]:
# Show top anomalies
top_anomalies = df[df['severity'].isin(['HIGH', 'MEDIUM'])].sort_values('anomaly_score', ascending=False).head(20)

print("\n🚨 Top 20 Detected Anomalies:")
display_cols = ['date', 'stock_name', 'close', 'volume', 'price_change', 'volume_zscore', 'severity', 'anomaly_score']
print(top_anomalies[display_cols].to_string())


🚨 Top 20 Detected Anomalies:
            date          stock_name  close  volume  price_change  volume_zscore severity  anomaly_score
2531  2023-01-06  ATTIJARI BANK       46.00   16086      0.179790       3.631483     HIGH            1.0
11091 2022-07-02  SOPAT                1.65   48886      0.213235       3.147484     HIGH            1.0
2994  2024-04-01  ATTIJARI LEASING    16.50    3101     -0.136126       4.096392     HIGH            1.0
15175 2025-08-01  TUNISIE LEASING F   17.98   25060     -0.457617       3.297481     HIGH            1.0
15238 2022-03-11  UIB                 23.82  204245      0.151281       4.219358     HIGH            1.0
8489  2025-05-05  OFFICEPLAST          1.75   34340      0.093750       4.219863     HIGH            1.0
8821  2024-10-10  ONE TECH HOLDING     9.88  388721      0.079781       3.477609     HIGH            1.0
11880 2025-02-05  SOTIPAPIER           3.72   49777     -0.160271       3.454119     HIGH            1.0
7652  2025-07-11  LAND OR

In [20]:
# Visualize anomalies for a sample stock
sample_stock = 'SFBT'
stock_data = df[df['stock_name'] == sample_stock].copy()

if len(stock_data) > 0:
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))

    # Price with anomalies
    ax1 = axes[0]
    ax1.plot(stock_data['date'], stock_data['close'], color='white', linewidth=1, label='Price')

    # Mark anomalies
    high_anomalies = stock_data[stock_data['severity'] == 'HIGH']
    medium_anomalies = stock_data[stock_data['severity'] == 'MEDIUM']

    ax1.scatter(high_anomalies['date'], high_anomalies['close'],
                color='#ff4444', s=100, label='HIGH Severity', zorder=5)
    ax1.scatter(medium_anomalies['date'], medium_anomalies['close'],
                color='#ffaa00', s=50, label='MEDIUM Severity', zorder=4)

    ax1.set_title(f'{sample_stock} - Price with Anomalies', color='white')
    ax1.legend()
    ax1.grid(alpha=0.3)

    # Volume with anomalies
    ax2 = axes[1]
    ax2.bar(stock_data['date'], stock_data['volume'], color='#444444', alpha=0.7, label='Volume')
    ax2.bar(high_anomalies['date'], high_anomalies['volume'], color='#ff4444', label='Spike')

    ax2.set_title(f'{sample_stock} - Volume with Spikes', color='white')
    ax2.legend()
    ax2.grid(alpha=0.3)

    plt.tight_layout()
    plt.show()

## 6. Export Model & Alert Generator

In [21]:
import joblib
import json
import os

os.makedirs('models', exist_ok=True)

# Save Isolation Forest model
joblib.dump(iso_forest, 'models/anomaly_detector.pkl')
print("✅ Saved models/anomaly_detector.pkl")

# Save scaler
joblib.dump(scaler, 'models/anomaly_scaler.pkl')
print("✅ Saved models/anomaly_scaler.pkl")

# Save config
anomaly_config = {
    'feature_columns': ANOMALY_FEATURES,
    'thresholds': {
        'volume_spike_zscore': 3.0,
        'price_change_pct': 0.05,
        'gap_open_pct': 0.03,
        'range_ratio': 2.0
    },
    'severity_levels': {
        'HIGH': 0.6,
        'MEDIUM': 0.3,
        'LOW': 0.0
    },
    'contamination': 0.02,
    'created_at': datetime.now().isoformat()
}

with open('models/anomaly_config.json', 'w') as f:
    json.dump(anomaly_config, f, indent=2)
print("✅ Saved models/anomaly_config.json")

✅ Saved models/anomaly_detector.pkl
✅ Saved models/anomaly_scaler.pkl
✅ Saved models/anomaly_config.json


In [ ]:
# Download models
!zip -r anomaly_models.zip models/
files.download('anomaly_models.zip')

## 7. Inference Class (for Backend)

In [ ]:
# === COPY THIS TO YOUR BACKEND ===

class BVMTAnomalyDetector:
    """
    Production-ready anomaly detection service for BVMT stocks.

    Combines rule-based thresholds with Isolation Forest for robust detection.
    """

    THRESHOLDS = {
        'volume_spike_zscore': 3.0,
        'price_change_pct': 0.05,
        'gap_open_pct': 0.03,
        'range_ratio': 2.0
    }

    FEATURE_COLS = [
        'volume_zscore', 'volume_ratio', 'price_change_abs', 'price_zscore',
        'intraday_range', 'range_ratio', 'gap_open_abs', 'tx_ratio', 'vol_price_ratio'
    ]

    def __init__(self, model_dir: str):
        self.model_dir = model_dir
        self.model = None
        self.scaler = None
        self._load_models()

    def _load_models(self):
        import joblib
        try:
            self.model = joblib.load(f'{self.model_dir}/anomaly_detector.pkl')
            self.scaler = joblib.load(f'{self.model_dir}/anomaly_scaler.pkl')
            print("✅ Anomaly detection models loaded")
        except Exception as e:
            print(f"⚠️ Could not load models: {e}")

    def detect(self, stock_name: str, current_data: dict, historical_stats: dict) -> dict:
        """
        Detect anomalies for a single data point.

        Args:
            stock_name: Name of stock
            current_data: Dict with keys [open, high, low, close, volume, transactions]
            historical_stats: Dict with keys [volume_ma_20, volume_std_20, price_ma_20, price_std_20, etc.]

        Returns:
            Anomaly detection result
        """
        # Calculate features
        features = self._calculate_features(current_data, historical_stats)

        # Rule-based detection
        alerts = []
        severity_score = 0

        if features['volume_zscore'] > self.THRESHOLDS['volume_spike_zscore']:
            alerts.append({
                'type': 'VOLUME_SPIKE',
                'message': f"Volume is {features['volume_zscore']:.1f}σ above average",
                'severity': 'HIGH' if features['volume_zscore'] > 5 else 'MEDIUM'
            })
            severity_score += 0.3

        if features['price_change_abs'] > self.THRESHOLDS['price_change_pct']:
            direction = 'up' if features.get('price_change', 0) > 0 else 'down'
            alerts.append({
                'type': 'PRICE_MOVE',
                'message': f"Price moved {features['price_change_abs']*100:.1f}% {direction}",
                'severity': 'HIGH' if features['price_change_abs'] > 0.10 else 'MEDIUM'
            })
            severity_score += 0.3

        if features['gap_open_abs'] > self.THRESHOLDS['gap_open_pct']:
            alerts.append({
                'type': 'GAP_OPEN',
                'message': f"Gap open of {features['gap_open_abs']*100:.1f}%",
                'severity': 'MEDIUM'
            })
            severity_score += 0.1

        # ML-based detection
        if self.model and self.scaler:
            X = np.array([[features.get(col, 0) for col in self.FEATURE_COLS]])
            X = np.nan_to_num(X, nan=0, posinf=0, neginf=0)
            X_scaled = self.scaler.transform(X)

            ml_prediction = self.model.predict(X_scaled)[0]
            ml_score = self.model.decision_function(X_scaled)[0]

            if ml_prediction == -1:  # Anomaly
                alerts.append({
                    'type': 'ML_ANOMALY',
                    'message': f"Unusual pattern detected (score: {ml_score:.3f})",
                    'severity': 'MEDIUM'
                })
                severity_score += 0.2

        # Determine overall severity
        if severity_score >= 0.6:
            overall_severity = 'HIGH'
        elif severity_score >= 0.3:
            overall_severity = 'MEDIUM'
        elif severity_score > 0:
            overall_severity = 'LOW'
        else:
            overall_severity = 'NONE'

        return {
            'stock': stock_name,
            'timestamp': datetime.now().isoformat(),
            'is_anomaly': len(alerts) > 0,
            'severity': overall_severity,
            'severity_score': round(severity_score, 2),
            'alerts': alerts,
            'features': {k: round(v, 4) if isinstance(v, float) else v for k, v in features.items()}
        }

    def _calculate_features(self, current: dict, historical: dict) -> dict:
        """Calculate anomaly detection features from current and historical data."""
        volume = current.get('volume', 0)
        close = current.get('close', 0)
        open_price = current.get('open', close)
        high = current.get('high', close)
        low = current.get('low', close)
        prev_close = historical.get('prev_close', close)

        vol_ma = historical.get('volume_ma_20', volume)
        vol_std = historical.get('volume_std_20', 1)
        price_ma = historical.get('price_ma_20', close)
        price_std = historical.get('price_std_20', 1)
        range_ma = historical.get('range_ma_10', 0.01)
        tx_ma = historical.get('tx_ma_20', 1)

        price_change = (close - prev_close) / prev_close if prev_close > 0 else 0
        intraday_range = (high - low) / close if close > 0 else 0
        gap_open = (open_price - prev_close) / prev_close if prev_close > 0 else 0

        return {
            'volume_zscore': (volume - vol_ma) / vol_std if vol_std > 0 else 0,
            'volume_ratio': volume / vol_ma if vol_ma > 0 else 1,
            'price_change': price_change,
            'price_change_abs': abs(price_change),
            'price_zscore': (close - price_ma) / price_std if price_std > 0 else 0,
            'intraday_range': intraday_range,
            'range_ratio': intraday_range / range_ma if range_ma > 0 else 1,
            'gap_open_abs': abs(gap_open),
            'tx_ratio': current.get('transactions', 0) / tx_ma if tx_ma > 0 else 1,
            'vol_price_ratio': (volume / vol_ma) / (abs(price_change) + 0.001) if vol_ma > 0 else 0
        }

print("✅ BVMTAnomalyDetector class ready!")

## Summary

### What We Built:
1. ✅ Rule-based detection (volume spikes, price moves, gaps)
2. ✅ Isolation Forest ML detection
3. ✅ Hybrid scoring with severity levels
4. ✅ Production-ready inference class

### Files to Copy:
```
ml/models/
├── anomaly_detector.pkl
├── anomaly_scaler.pkl
└── anomaly_config.json
```

### Backend Integration:
Copy `BVMTAnomalyDetector` class to `backend/app/services/anomaly.py`